![](http://www.timzhangyuxuan.com/static/images/project_DCGAN/structure.png)

- The generator and discriminator are trying to optimize the opposite thing

### Discriminator Cost(Binary CrossEntropy)

\begin{equation*}
J= -[t logy +(1-t)log(1-y)]
\end{equation*}

- t = 1 mena real and t = 0 mena fake


- z is latent vector G(z) is generator output function

\begin{equation*}
J^{(D)}= -[logD(x) +log(1-D(G(z)))]  
\end{equation*}
\begin{equation*}
J^D(θ^D, θ^G) = − \frac{1}{2} \mathbb{E}_{x \sim p_{data}} log D(x) − \frac{1}{2} \mathbb{E}_{z} log  (1 − D (G(z)))
\end{equation*}


### Generator Cost

- try to maximize the discriminator's cost


\begin{equation*}
J^{G}= -J^{D}
\end{equation*}


### Minimax

- zero-sum game are called minimax game


\begin{equation*}
\theta_{G}^{*} = argmin_{\theta_{G}}max_{\theta_{D}}- J^{D}
\end{equation*}


![](https://jhui.github.io/assets/gm/gmc.png)

$log(1-D(G(z)))]  $ , if $D(G(z)$ is 1, slope is $-\infty$  the gradient of J diminishes for MiniMax game and Maximum likelihood game. The generator cannot learn.


## DCGAN

### Generator

![](https://jhui.github.io/assets/gm/gm.png)

### Input Transpose convolution

![](https://jhui.github.io/assets/gm/transpose.gif)







#### Fractional strided convolution

- if we use stride = 1/2, result is 2x original

- in TF, convolution filter is specified like  
(filter height, filter width, # feature map in, # feature map out)
- Strides   
(1, vertical stride, horizontal stride,1)

- here  
(filter height, filter width, # feature map out,# feature map in)



- The input z to the model is a 100-Dimensional vector (100 random numbers)

#### Batch Normalization
- with batch norm, instead of manually normalizing data first, we do a normalization at every layer of the neural net



$X_{B}$ = next batch of data  
$\mu_{B}$ = mean($X_{B}$)  
$\sigma$ = std($X_{B}$)  
$Y_{B}$ = ($X_{B}-\mu_{B}$)\$\sigma_{B}$   

$Y_{B}$ = ($X_{B}-\mu_{B}$)\ $\sqrt{\sigma_{B}^2+\epsilon}$

### - MNIST dataset

```python
class DCGAN:
  def __init__(self, img_length, num_colors, d_sizes, g_sizes):

    # save for later
    self.img_length = img_length # 28
    self.num_colors = num_colors # 1
    self.latent_dims = g_sizes['z'] # 100

    '(N,28,28,1)'
    # define the input data
    self.X = tf.placeholder(
      tf.float32,
      shape=(None, img_length, img_length, num_colors),
      name='X'
    )
    '(N,100)'
    self.Z = tf.placeholder(
      tf.float32,
      shape=(None, self.latent_dims),
      name='Z'
    )

    self.batch_sz = tf.placeholder(tf.int32, shape=(), name='batch_sz')
    
    # build the discriminator
    logits = self.build_discriminator(self.X, d_sizes)

    # build generator
    self.sample_images = self.build_generator(self.Z, g_sizes)
    ...
    
    def build_generator(self, Z, g_sizes):
        with tf.variable_scope("generator") as scope:

            'dims = [28,14,7]' #stride is 2  
            # determine the size of the data at each step
            dims = [self.img_length]
            dim = self.img_length
            for _, _, stride, _ in reversed(g_sizes['conv_layers']):
                dim = int(np.ceil(float(dim) / stride))
                dims.append(dim)
            # note: dims is actually backwards
            # the first layer of the generator is actually last
            # so let's reverse it
            dims = list(reversed(dims))
            print("dims:", dims)
            self.g_dims = dims
            'dims: [7, 14, 28]'
            
            '(N,100)->(N,1024)->(N,7,7,128)->(N,14,14,128)->(N,28,28,1)'
            'dense_layer shape g_denselayer_0 : (# latent input,#feature map out)'
            '(100,1024)'
            # dense layers
            mi = self.latent_dims
            self.g_denselayers = []
            count = 0
            for mo, apply_batch_norm in g_sizes['dense_layers']:
                name = "g_denselayer_%s" % count
                count += 1

                layer = DenseLayer(name, mi, mo, apply_batch_norm)
                self.g_denselayers.append(layer)
                mi = mo
            
            'mo = 128*7*7'
            'mi = 1024, mo= 128*7*7(6272)'
            # final dense layer
            mo = g_sizes['projection'] * dims[0] * dims[0]
            name = "g_denselayer_%s" % count
            layer = DenseLayer(name, mi, mo, not g_sizes['bn_after_project'])
            self.g_denselayers.append(layer)    
            
            'mi is 128'
            # fs-conv layers
            mi = g_sizes['projection']
            self.g_convlayers = []
            
            'num_relus is 1'
            'activation_functions = [tf.nn.relu, tf.sigmoid]
            # output may use tanh or sigmoid
            num_relus = len(g_sizes['conv_layers']) - 1
            activation_functions = [tf.nn.relu]*num_relus + [g_sizes['output_activation']]
            
            'output shape 0 (#batch num, 14,14,128)'
            'output shape 1 (#batch num, 28,28,1)'
            'mi: 128 mo: 128 outp shape: [<tf.Tensor 'batch_sz:0' shape=() dtype=int32>, 14, 14, 128]'
            'mi: 128 mo: 1 outp shape: [<tf.Tensor 'batch_sz:0' shape=() dtype=int32>, 28, 28, 1]'
            for i in range(len(g_sizes['conv_layers'])):
                name = "fs_convlayer_%s" % i
                mo, filtersz, stride, apply_batch_norm = g_sizes['conv_layers'][i]
                f = activation_functions[i]
                output_shape = [self.batch_sz, dims[i+1], dims[i+1], mo]
                print("mi:", mi, "mo:", mo, "outp shape:", output_shape)
                layer = FractionallyStridedConvLayer(
                  name, mi, mo, output_shape, apply_batch_norm, filtersz, stride, f
                )
                self.g_convlayers.append(layer)
                mi = mo
            # get the output
            self.g_sizes = g_sizes
            return self.g_forward(Z)
        
    def g_forward(self, Z, reuse=None, is_training=True):
        # dense layers
        output = Z
        for layer in self.g_denselayers:
            output = layer.forward(output, reuse, is_training)

        # project and reshape
        output = tf.reshape(
          output,
          [-1, self.g_dims[0], self.g_dims[0], self.g_sizes['projection']],
        )

        # apply batch norm
        if self.g_sizes['bn_after_project']:
            output = tf.contrib.layers.batch_norm(
                output,
                decay=0.9, 
                updates_collections=None,
                epsilon=1e-5,
                scale=True,
                is_training=is_training,
                reuse=reuse,
                scope='bn_after_project'
              )
        
        # pass through fs-conv layers
        for layer in self.g_convlayers:
            output = layer.forward(output, reuse, is_training)

        return output            
```

### Discriminator

![](https://image.slidesharecdn.com/unsupervisedlearningrepresenationwithdcgan-170608192900/95/unsupervised-learning-represenation-with-dcgan-14-638.jpg?cb=1496950222)

```txt
ConvLayer Tensor("discriminator/Maximum:0", shape=(?, 14, 14, 2), dtype=float32)
ConvLayer Tensor("discriminator/Maximum_1:0", shape=(?, 7, 7, 64), dtype=float32)
ConvLayer flatten Tensor("discriminator/Flatten/flatten/Reshape:0", shape=(?, 3136), dtype=float32)
Tensor("discriminator/add:0", shape=(?, 1024), dtype=float32)
denseLayer Tensor("discriminator/Maximum_2:0", shape=(?, 1024), dtype=float32)
Tensor("discriminator/add_1:0", shape=(?, 1), dtype=float32)
denseLayer logits Tensor("discriminator/add_1:0", shape=(?, 1), dtype=float32)
```

```python
# for mnist
d_sizes = {
'conv_layers': [(2, 5, 2, False), (64, 5, 2, True)],'mo, filtersz, stride, apply_batch_norm'
'dense_layers': [(1024, True)],'mo, apply_batch_norm'
}

# build the discriminator
logits = self.build_discriminator(self.X, d_sizes)

def build_discriminator(self, X, d_sizes):
    with tf.variable_scope("discriminator") as scope:

        '(N,28,28,1) ->(N,14,14,2):stride(2)->(N,7,7,64):stride(2)->flatten(N,3136)'
        # build conv layers
        self.d_convlayers = []
        mi = self.num_colors # 1
        dim = self.img_length # 28
        count = 0
        
        for mo, filtersz, stride, apply_batch_norm in d_sizes['conv_layers']:
            # make up a name - used for get_variable
            name = "convlayer_%s" % count
            count += 1

            layer = ConvLayer(name, mi, mo, apply_batch_norm, filtersz, stride, lrelu)
            self.d_convlayers.append(layer)
            mi = mo
            print("dim:", dim)
            dim = int(np.ceil(float(dim) / stride))
        
        'mi = mi * dim * dim= 64*7*7=3136 - > dense layer (N,1024)'
        mi = mi * dim * dim
        # build dense layers
        self.d_denselayers = []
        for mo, apply_batch_norm in d_sizes['dense_layers']:
            name = "denselayer_%s" % count
            count += 1

            layer = DenseLayer(name, mi, mo, apply_batch_norm, lrelu)
            mi = mo
            self.d_denselayers.append(layer)
        
        '->(N,1)'
        # final logistic layer
        name = "denselayer_%s" % count
        self.d_finallayer = DenseLayer(name, mi, 1, False, lambda x: x)

        


```

### Loss function



```python
# build the discriminator
logits = self.build_discriminator(self.X, d_sizes)

# build generator
self.sample_images = self.build_generator(self.Z, g_sizes)

# get sample logits
with tf.variable_scope("discriminator") as scope:
  scope.reuse_variables()
  sample_logits = self.d_forward(self.sample_images, True)

# get sample images for test time (batch norm is different)
with tf.variable_scope("generator") as scope:
  scope.reuse_variables()
  self.sample_images_test = self.g_forward(
    self.Z, reuse=True, is_training=False
  )

    
# build costs
self.d_cost_real = tf.nn.sigmoid_cross_entropy_with_logits(
  logits=logits,
  labels=tf.ones_like(logits)
)
self.d_cost_fake = tf.nn.sigmoid_cross_entropy_with_logits(
  logits=sample_logits,
  labels=tf.zeros_like(sample_logits)
)    

self.d_cost = tf.reduce_mean(self.d_cost_real) + tf.reduce_mean(self.d_cost_fake)

self.g_cost = tf.reduce_mean(
  tf.nn.sigmoid_cross_entropy_with_logits(
    logits=sample_logits,
    labels=tf.ones_like(sample_logits)
  )
)

real_predictions = tf.cast(logits > 0, tf.float32)
fake_predictions = tf.cast(sample_logits < 0, tf.float32)
num_predictions = 2.0*BATCH_SIZE
num_correct = tf.reduce_sum(real_predictions) + tf.reduce_sum(fake_predictions)
self.d_accuracy = num_correct / num_predictions


# optimizers
self.d_params = [t for t in tf.trainable_variables() if t.name.startswith('d')]#discriminator scope
self.g_params = [t for t in tf.trainable_variables() if t.name.startswith('g')]#generator scope

self.d_train_op = tf.train.AdamOptimizer(
  LEARNING_RATE, beta1=BETA1
).minimize(
  self.d_cost, var_list=self.d_params
)
self.g_train_op = tf.train.AdamOptimizer(
  LEARNING_RATE, beta1=BETA1
).minimize(
  self.g_cost, var_list=self.g_params
)


```

- reference  
[Generative-adversarial-models](https://jhui.github.io/2017/03/05/Generative-adversarial-models/)

In [2]:
import sys
sys.path.append("./opensrc/machine_learning_examples-master/unsupervised_class3") 

import util
import os
import util
import scipy as sp
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime



# X, Y = util.get_mnist()

In [3]:
import dcgan_tf as dcgan

X, Y = util.get_mnist()
X = X.reshape(len(X), 28, 28, 1)
dim = X.shape[1]
colors = X.shape[-1]

# for mnist
d_sizes = {
'conv_layers': [(2, 5, 2, False), (64, 5, 2, True)],
'dense_layers': [(1024, True)],
}
g_sizes = {
'z': 100,
'projection': 128,
'bn_after_project': False,
'conv_layers': [(128, 5, 2, True), (colors, 5, 2, False)],
'dense_layers': [(1024, True)],
'output_activation': tf.sigmoid,
}


# setup gan
# note: assume square images, so only need 1 dim
gan = dcgan.DCGAN(dim, colors, d_sizes, g_sizes)

Looks like you haven't downloaded the data or it's not in the right spot.
Please get train.csv from https://www.kaggle.com/c/digit-recognizer
and place it in the large_files folder.
Reading in and transforming data...
dim: 28
dim: 14
ConvLayer Tensor("discriminator/Maximum:0", shape=(?, 14, 14, 2), dtype=float32)
ConvLayer Tensor("discriminator/Maximum_1:0", shape=(?, 7, 7, 64), dtype=float32)
ConvLayer flatten Tensor("discriminator/Flatten/flatten/Reshape:0", shape=(?, 3136), dtype=float32)
Tensor("discriminator/add:0", shape=(?, 1024), dtype=float32)
denseLayer Tensor("discriminator/Maximum_2:0", shape=(?, 1024), dtype=float32)
Tensor("discriminator/add_1:0", shape=(?, 1), dtype=float32)
denseLayer logits Tensor("discriminator/add_1:0", shape=(?, 1), dtype=float32)
dims: [7, 14, 28]
mi: 128 mo: 128 outp shape: [<tf.Tensor 'batch_sz:0' shape=() dtype=int32>, 14, 14, 128]
mi: 128 mo: 1 outp shape: [<tf.Tensor 'batch_sz:0' shape=() dtype=int32>, 28, 28, 1]
Tensor("generator/add:0", shap

In [16]:
len(g_sizes)

6